In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import anndata as ad
import sys
import bbknn
from statsmodels import robust
import matplotlib.pyplot as plt
import os.path
import scirpy as ir
from matplotlib import pyplot as plt, cm as mpl_cm
from cycler import cycler
import os

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
#sc.settings.set_figure_params(dpi=80, color_map='viridis')
#sc.logging.print_versions()

In [2]:
path = "/lustre/scratch126/cellgen/team205/rl20/T_cell_lym/SoupX_out"
manifest = pd.read_csv(os.path.join(path, 'sample_manifest.csv'))
manifest

,sample_id,donor_id,tumour_type,age,site,type
0,TL10712_CD3pos_GEX_CITE,TL10712,AITL,Unknown,Axilliary,primary
1,TL10712_CD3neg_GEX,TL10712,AITL,Unknown,Axilliary,primary
2,TL2860_CD3neg_GEX,TL2860,TLBL,elderly,Unknown,primary
3,TL2860_CD3pos_GEX_CITE,TL2860,TLBL,elderly,Unknown,primary
4,TL3291_GEX_CITE,TL3291,TLBL,Unknown,Cervical,primary
5,TL4528_CD3neg_GEX,TL4528,TCL,57,Mesenteric,primary
6,TL4528_CD3pos_GEX_CITE,TL4528,TCL,57,Mesenteric,primary
7,TL5493_CD3neg_GEX,TL5493,PTCL,69,Cervical,primary
8,TL5493_CD3pos_GEX_CITE,TL5493,PTCL,69,Cervical,primary
9,TL5521_GEX_CITE,TL5521,TLBL,17,Left_inguinal,primary


In [3]:
names = manifest['sample_id'].tolist()
filenames = [os.path.join(path, n+'_soupx_out') for n in names]
filenames

['/lustre/scratch126/cellgen/team205/rl20/T_cell_lym/SoupX_out/TL10712_CD3pos_GEX_CITE_soupx_out',
 '/lustre/scratch126/cellgen/team205/rl20/T_cell_lym/SoupX_out/TL10712_CD3neg_GEX_soupx_out',
 '/lustre/scratch126/cellgen/team205/rl20/T_cell_lym/SoupX_out/TL2860_CD3neg_GEX_soupx_out',
 '/lustre/scratch126/cellgen/team205/rl20/T_cell_lym/SoupX_out/TL2860_CD3pos_GEX_CITE_soupx_out',
 '/lustre/scratch126/cellgen/team205/rl20/T_cell_lym/SoupX_out/TL3291_GEX_CITE_soupx_out',
 '/lustre/scratch126/cellgen/team205/rl20/T_cell_lym/SoupX_out/TL4528_CD3neg_GEX_soupx_out',
 '/lustre/scratch126/cellgen/team205/rl20/T_cell_lym/SoupX_out/TL4528_CD3pos_GEX_CITE_soupx_out',
 '/lustre/scratch126/cellgen/team205/rl20/T_cell_lym/SoupX_out/TL5493_CD3neg_GEX_soupx_out',
 '/lustre/scratch126/cellgen/team205/rl20/T_cell_lym/SoupX_out/TL5493_CD3pos_GEX_CITE_soupx_out',
 '/lustre/scratch126/cellgen/team205/rl20/T_cell_lym/SoupX_out/TL5521_GEX_CITE_soupx_out',
 '/lustre/scratch126/cellgen/team205/rl20/T_cell_lym

In [5]:
adatas = [sc.read_10x_mtx(filename, cache=True) for filename in filenames]


... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedup reading next time
... writing an h5ad cache file to speedu

#### TCR (use dandelion now)

In [5]:
path = "/lustre/scratch126/cellgen/team205/rl20/T_cell_lym"
manifest_tcr = pd.read_csv(os.path.join(path, 'tcr_sample_manifest.csv'))
manifest_tcr

,sample_id,donor_id,gex_id
0,TL10712_TCR,TL10712,TL10712_CD3pos_GEX_CITE
1,TL2860_TCR,TL2860,TL2860_CD3pos_GEX_CITE
2,TL3291_TCR,TL3291,TL3291_GEX_CITE
3,TL4528_TCR,TL4528,TL4528_CD3pos_GEX_CITE
4,TL5493_TCR,TL5493,TL5493_CD3pos_GEX_CITE
5,TL5521_TCR,TL5521,TL5521_GEX_CITE
6,TL5737_TCR,TL5737,TL5737_GEX_CITE
7,TL5968_TCR,TL5968,TL5968_GEX_CITE
8,TL6042_TCR,TL6042,TL6042_CD3pos_GEX_CITE
9,TL6678_TCR,TL6678,TL6678_CD3pos_GEX_CITE


In [6]:
names_tcr = manifest_tcr['sample_id'].tolist()
filenames_tcr = [os.path.join(path, n, 'filtered_contig_annotations.csv') for n in names_tcr]
filenames_tcr

['/lustre/scratch117/cellgen/team205/rl20/T_cell_lym/TL10712_TCR/filtered_contig_annotations.csv',
 '/lustre/scratch117/cellgen/team205/rl20/T_cell_lym/TL2860_TCR/filtered_contig_annotations.csv',
 '/lustre/scratch117/cellgen/team205/rl20/T_cell_lym/TL3291_TCR/filtered_contig_annotations.csv',
 '/lustre/scratch117/cellgen/team205/rl20/T_cell_lym/TL4528_TCR/filtered_contig_annotations.csv',
 '/lustre/scratch117/cellgen/team205/rl20/T_cell_lym/TL5493_TCR/filtered_contig_annotations.csv',
 '/lustre/scratch117/cellgen/team205/rl20/T_cell_lym/TL5521_TCR/filtered_contig_annotations.csv',
 '/lustre/scratch117/cellgen/team205/rl20/T_cell_lym/TL5737_TCR/filtered_contig_annotations.csv',
 '/lustre/scratch117/cellgen/team205/rl20/T_cell_lym/TL5968_TCR/filtered_contig_annotations.csv',
 '/lustre/scratch117/cellgen/team205/rl20/T_cell_lym/TL6042_TCR/filtered_contig_annotations.csv',
 '/lustre/scratch117/cellgen/team205/rl20/T_cell_lym/TL6678_TCR/filtered_contig_annotations.csv',
 '/lustre/scratch11

In [7]:
adatas_tcr = [ir.io.read_10x_vdj(filename) for filename in filenames_tcr]


In [8]:
for i in range(len(adatas_tcr)):
    ir.tl.chain_qc(adatas_tcr[i])
    

In [9]:
ir.pp.merge_with_ir(adatas[0], adatas_tcr[0])
ir.pp.merge_with_ir(adatas[3], adatas_tcr[1])
ir.pp.merge_with_ir(adatas[4], adatas_tcr[2])
ir.pp.merge_with_ir(adatas[6], adatas_tcr[3])
ir.pp.merge_with_ir(adatas[8], adatas_tcr[4])
ir.pp.merge_with_ir(adatas[9], adatas_tcr[5])
ir.pp.merge_with_ir(adatas[10], adatas_tcr[6])
ir.pp.merge_with_ir(adatas[11], adatas_tcr[7])
ir.pp.merge_with_ir(adatas[13], adatas_tcr[8])
ir.pp.merge_with_ir(adatas[15], adatas_tcr[9])
ir.pp.merge_with_ir(adatas[17], adatas_tcr[10])
ir.pp.merge_with_ir(adatas[18], adatas_tcr[11])
ir.pp.merge_with_ir(adatas[20], adatas_tcr[12])
ir.pp.merge_with_ir(adatas[21], adatas_tcr[13])
ir.pp.merge_with_ir(adatas[22], adatas_tcr[14])
ir.pp.merge_with_ir(adatas[23], adatas_tcr[15])
ir.pp.merge_with_ir(adatas[24], adatas_tcr[16])


... storing 'receptor_type' as categorical
... storing 'receptor_subtype' as categorical
... storing 'chain_pairing' as categorical
... storing 'receptor_type' as categorical
... storing 'receptor_subtype' as categorical
... storing 'chain_pairing' as categorical
... storing 'receptor_type' as categorical
... storing 'receptor_subtype' as categorical
... storing 'chain_pairing' as categorical
... storing 'receptor_type' as categorical
... storing 'receptor_subtype' as categorical
... storing 'chain_pairing' as categorical
... storing 'receptor_type' as categorical
... storing 'receptor_subtype' as categorical
... storing 'chain_pairing' as categorical
... storing 'receptor_type' as categorical
... storing 'receptor_subtype' as categorical
... storing 'chain_pairing' as categorical
... storing 'receptor_type' as categorical
... storing 'receptor_subtype' as categorical
... storing 'chain_pairing' as categorical
... storing 'receptor_type' as categorical
... storing 'receptor_subtype' as

In [6]:
#ID = manifest['sample_id'].tolist()
#ID = list(map(str, ID))
#ID = ['_'.join(n.split('/')[8].split('_')[0:2]) for n in filenames]
ID = [n.split('/')[8].split('_soupx_out')[0] for n in filenames]
#ID
adata = adatas[0].concatenate(adatas[1:], join = 'inner', index_unique='-', batch_key='ID', batch_categories=ID)


In [7]:
adata.obs

,ID
AAACCTGAGCCGTCGT-1-TL10712_CD3pos_GEX_CITE,TL10712_CD3pos_GEX_CITE
AAACCTGAGGATATAC-1-TL10712_CD3pos_GEX_CITE,TL10712_CD3pos_GEX_CITE
AAACCTGAGTTCGCAT-1-TL10712_CD3pos_GEX_CITE,TL10712_CD3pos_GEX_CITE
AAACCTGCAAGAAAGG-1-TL10712_CD3pos_GEX_CITE,TL10712_CD3pos_GEX_CITE
AAACCTGCAGCATGAG-1-TL10712_CD3pos_GEX_CITE,TL10712_CD3pos_GEX_CITE
...,...
TTTGTCATCACGCGGT-1-TL_SJH03_GEX_CITE,TL_SJH03_GEX_CITE
TTTGTCATCGGCATCG-1-TL_SJH03_GEX_CITE,TL_SJH03_GEX_CITE
TTTGTCATCTGGTGTA-1-TL_SJH03_GEX_CITE,TL_SJH03_GEX_CITE
TTTGTCATCTGTCTCG-1-TL_SJH03_GEX_CITE,TL_SJH03_GEX_CITE


In [8]:
manifest.index = manifest['sample_id']
manifest


,sample_id,donor_id,tumour_type,age,site,type
sample_id,,,,,,
TL10712_CD3pos_GEX_CITE,TL10712_CD3pos_GEX_CITE,TL10712,AITL,Unknown,Axilliary,primary
TL10712_CD3neg_GEX,TL10712_CD3neg_GEX,TL10712,AITL,Unknown,Axilliary,primary
TL2860_CD3neg_GEX,TL2860_CD3neg_GEX,TL2860,TLBL,elderly,Unknown,primary
TL2860_CD3pos_GEX_CITE,TL2860_CD3pos_GEX_CITE,TL2860,TLBL,elderly,Unknown,primary
TL3291_GEX_CITE,TL3291_GEX_CITE,TL3291,TLBL,Unknown,Cervical,primary
TL4528_CD3neg_GEX,TL4528_CD3neg_GEX,TL4528,TCL,57,Mesenteric,primary
TL4528_CD3pos_GEX_CITE,TL4528_CD3pos_GEX_CITE,TL4528,TCL,57,Mesenteric,primary
TL5493_CD3neg_GEX,TL5493_CD3neg_GEX,TL5493,PTCL,69,Cervical,primary
TL5493_CD3pos_GEX_CITE,TL5493_CD3pos_GEX_CITE,TL5493,PTCL,69,Cervical,primary


In [9]:
manifest = manifest.loc[ID,:]


In [10]:
#%% Metadata
for metadata in manifest.columns[0:]:
    print(metadata)
    adata.obs[metadata] = adata.obs['ID']
    replacement = manifest[metadata].tolist()
    adata.obs[metadata].replace(to_replace=ID, value=replacement, inplace=True)

sample_id
donor_id
tumour_type
age
site
type


In [11]:
#adata[adata.obs['receptor_subtype']=='TRA+TRB'].obs['donor_id'].value_counts()


In [16]:
adata[adata.obs['receptor_subtype']=='TRA+TRB'].obs['ID'].value_counts()


TL5521_GEX_CITE                      14839
TL6678_CD3pos_GEX_CITE                8916
TL5493_CD3pos_GEX_CITE                6665
TL10712_CD3pos_GEX_CITE               3354
TL6682_CD3pos_GEX_CITE                3335
TL6042_CD3pos_GEX_CITE                2912
TL3291_CD3pos_GEX_CITE                2891
TL2860_CD3pos_GEX_CITE                2873
TL8441_CD3pos_GEX_CITE                2433
TLPDX_GEX_CITE                        1234
TL6722_GEX_CITE                       1155
TL4528_CD3pos_GEX_CITE                1135
TL5737_GEX_CITE                        608
TL3291_GEX_CITE                        450
TL5737_CD3pos_GEX_CITE                 266
TL5968_GEX_CITE                        124
TL_Mangosteen_PDX_CD3pos_GEX_CITE      110
Name: ID, dtype: int64

In [12]:
save_file = '/lustre/scratch126/cellgen/team205/rl20/T_cell_lym/object/TCL_final_raw_soupX.h5ad'
adata.write(save_file)


... storing 'sample_id' as categorical
... storing 'donor_id' as categorical
... storing 'tumour_type' as categorical
... storing 'age' as categorical
... storing 'site' as categorical
... storing 'type' as categorical


#### Doublet removal--scrublet

In [13]:
adata

AnnData object with n_obs × n_vars = 211634 × 36601
    obs: 'ID', 'sample_id', 'donor_id', 'tumour_type', 'age', 'site', 'type'
    var: 'gene_ids'

In [14]:
meta_10x_channels = 'sample_id'


In [15]:
RUNs, DSs, CELLs, THRs, MEDs, MADs, CUTs, no_thr = [], [], [], [], [], [], [], []

for run in adata.obs[meta_10x_channels].unique():
    print(run)
    ad = adata[adata.obs[meta_10x_channels] == run, :]
    x = ad.X
    scrub = scr.Scrublet(x)
    ds, prd = scrub.scrub_doublets()
    RUNs.append(run)
    DSs.append(ds)
    CELLs.append(ad.obs_names)
    # MAD calculation of threshold:
    MED = np.median(ds)
    MAD = robust.mad(ds)
    CUT = (MED + (3 * MAD))
    MEDs.append(MED)
    MADs.append(MAD)
    CUTs.append(CUT)

    try:  # not always can calculate automatic threshold
        THRs.append(scrub.threshold_)
        print('Threshold found by scrublet')
    except:
        THRs.append(0.4)
        no_thr.append(run)
        print('No threshold found, assigning 0.4 to', run)
        scrub.call_doublets(threshold=0.4) # so that it can make the plot
    fig = scrub.plot_histogram()
    fig[0].savefig('figures/'+run + '.png')

    # Alternative histogram for MAD-based cutoff
    scrub.call_doublets(threshold=CUT)
    fig = scrub.plot_histogram()
    fig[0].savefig(run + '_mad_' + '.png')
    plt.close('all')
    print()
    print()

TL10712_CD3pos_GEX_CITE
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.75
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.5%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 5.6%
Elapsed time: 15.2 seconds
Threshold found by scrublet
Detected doublet rate = 7.2%
Estimated detectable doublet fraction = 40.8%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 17.6%


TL10712_CD3neg_GEX
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.34
Detected doublet rate = 5.5%
Estimated detectable doublet fraction = 33.3%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 16.5%
Elapsed time: 11.3 seconds
Threshold found by scrublet
Detected doublet rate = 4.5%
Estimated detectable doublet fraction = 29.8%
Overall doublet rate:
	Expected   = 10.0%

In [16]:
ns = np.array(list(map(len, DSs)))


In [17]:
tbl = pd.DataFrame({
    'run': np.repeat(RUNs, ns),
    'ds': np.concatenate(DSs),
    'thr': np.repeat(THRs, ns),
    'mad_MED': np.repeat(MEDs, ns),
    'mad_MAD': np.repeat(MADs, ns),
    'mad_thr': np.repeat(CUTs, ns),
    }, index=np.concatenate(CELLs))

tbl['auto_prd'] = tbl['ds'] > tbl['thr']
tbl['mad_prd'] = tbl['ds'] > tbl['mad_thr']

In [18]:
adata.obs['mad_prd']=tbl['mad_prd']
adata.obs['ds']=tbl['ds']
adata.obs['mad_MED']=tbl['mad_MED']
adata.obs['mad_MAD']=tbl['mad_MAD']
adata.obs['mad_thr']=tbl['mad_thr']
adata = adata[tbl['mad_prd'] != True]


In [19]:
adata.obs


,ID,sample_id,donor_id,tumour_type,age,site,type,mad_prd,ds,mad_MED,mad_MAD,mad_thr
AAACCTGAGCCGTCGT-1-TL10712_CD3pos_GEX_CITE,TL10712_CD3pos_GEX_CITE,TL10712_CD3pos_GEX_CITE,TL10712,AITL,Unknown,Axilliary,primary,False,0.072886,0.075099,0.042061,0.201282
AAACCTGAGGATATAC-1-TL10712_CD3pos_GEX_CITE,TL10712_CD3pos_GEX_CITE,TL10712_CD3pos_GEX_CITE,TL10712,AITL,Unknown,Axilliary,primary,False,0.072886,0.075099,0.042061,0.201282
AAACCTGAGTTCGCAT-1-TL10712_CD3pos_GEX_CITE,TL10712_CD3pos_GEX_CITE,TL10712_CD3pos_GEX_CITE,TL10712,AITL,Unknown,Axilliary,primary,False,0.052632,0.075099,0.042061,0.201282
AAACCTGCAAGAAAGG-1-TL10712_CD3pos_GEX_CITE,TL10712_CD3pos_GEX_CITE,TL10712_CD3pos_GEX_CITE,TL10712,AITL,Unknown,Axilliary,primary,False,0.031229,0.075099,0.042061,0.201282
AAACCTGCAGCATGAG-1-TL10712_CD3pos_GEX_CITE,TL10712_CD3pos_GEX_CITE,TL10712_CD3pos_GEX_CITE,TL10712,AITL,Unknown,Axilliary,primary,False,0.082206,0.075099,0.042061,0.201282
...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCAGTTTCCACC-1-TL_SJH03_GEX_CITE,TL_SJH03_GEX_CITE,TL_SJH03_GEX_CITE,TL_SJH03,BIA-ALCL,Unknown,Breast,primary,False,0.063800,0.092308,0.062445,0.279643
TTTGTCATCACGCGGT-1-TL_SJH03_GEX_CITE,TL_SJH03_GEX_CITE,TL_SJH03_GEX_CITE,TL_SJH03,BIA-ALCL,Unknown,Breast,primary,False,0.053232,0.092308,0.062445,0.279643
TTTGTCATCTGGTGTA-1-TL_SJH03_GEX_CITE,TL_SJH03_GEX_CITE,TL_SJH03_GEX_CITE,TL_SJH03,BIA-ALCL,Unknown,Breast,primary,False,0.268022,0.092308,0.062445,0.279643
TTTGTCATCTGTCTCG-1-TL_SJH03_GEX_CITE,TL_SJH03_GEX_CITE,TL_SJH03_GEX_CITE,TL_SJH03,BIA-ALCL,Unknown,Breast,primary,False,0.130901,0.092308,0.062445,0.279643


In [20]:
#adata[adata.obs['receptor_subtype']=='TRA+TRB'].obs['donor_id'].value_counts()

In [21]:
save_file = '/lustre/scratch126/cellgen/team205/rl20/T_cell_lym/object/TCL_final_raw_soupX_rmdou.h5ad'
adata.write(save_file)